### generate_synthetic_principles_for_agents.ipynb
This notebook generates an agents synthetic principles for use in HCVA and stores these in a `.csv` file
- Takes an individual agents value system
    - finds the decision they would take, and assigns the agent an ethical principle that fits with their view, or is the opposite of their view according to some probability $p$

In [26]:
import pandas as pd
import numpy as np

#####
# 08/01/2025: 1.0 = Utilitarian, 1.4 = Transition point, 10.0 = Egalitarian
###
relevent_consensuses = [round(x * 0.1, 1) for x in range(10, 101)]
relevant_columns = ['p','Rel-Nonrel', 'Nonrel-Rel', 'Rel_div_p', 'Nonrel_div_p', 'Egal-Util', 'Util-Egal']

actions_filename = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/single_example_results/single_example/08-01-2025-actions.csv"
preference_filename = '/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/single_example_results/single_example/08-01-2025-preferences.csv'

final_df_savename = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/10-01-2025-ess-relevant-consensus.csv"

pref_df = pd.read_csv(preference_filename)
act_df = pd.read_csv(actions_filename)
cons_df = pd.merge(pref_df, act_df, on='p')
cons_df['p'] = cons_df['p'].round(1)

In [27]:
cons_df

,p,Up_x,Dist1_x,Distl_x,Rel-Rel,Rel-Nonrel,Nonrel-Rel,Nonrel-Nonrel,Up_y,Dist1_y,Distl_y,Rel_div_p,Nonrel_div_p,Rel_div_n,Nonrel_div_n
0,1.0,4.278681,0.000000,0.032110,0.0,0.346411,0.653589,0.0,9.573980,0.000000,0.042509,0.001517,0.028995,-0.001517,-0.028995
1,1.1,3.130946,0.000691,0.060989,0.0,0.346043,0.653957,0.0,6.586631,0.015874,0.095706,-0.006315,0.028138,0.006315,-0.028138
2,1.2,2.422156,0.001092,0.058305,0.0,0.345798,0.654202,0.0,4.832512,0.036973,0.063924,-0.014573,0.022175,0.014573,-0.022175
3,1.3,1.955035,0.001220,0.055947,0.0,0.345695,0.654305,0.0,3.728159,0.044339,0.048547,-0.018456,0.018935,0.018456,-0.018935
4,1.4,1.631161,0.001041,0.053723,0.0,0.345776,0.654224,0.0,2.992951,0.046732,0.039915,-0.020717,0.017142,0.020717,-0.017142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,9.7,0.310001,0.027084,0.007404,0.0,0.371627,0.628373,0.0,0.379555,0.057535,0.011877,-0.052050,0.010659,0.052050,-0.010659
88,9.8,0.309425,0.027161,0.007302,0.0,0.371717,0.628283,0.0,0.378643,0.057583,0.011959,-0.052134,0.010613,0.052134,-0.010613
89,9.9,0.308863,0.027237,0.007202,0.0,0.371806,0.628194,0.0,0.377755,0.057630,0.012038,-0.052216,0.010568,0.052216,-0.010568
90,10.0,0.308313,0.027311,0.007103,0.0,0.371893,0.628107,0.0,0.376890,0.057674,0.012114,-0.052295,0.010524,0.052295,-0.010524


In [28]:
# for each row, find the corresponding Util-Egal and Egal-Util values (where P=1.0 has Util-Egal=1.0 and Egal-Util=0.0, and P=10.0 has Util-Egal=0.0 and Egal-Util=1.0)
for index, row in cons_df.iterrows():
    cons_df.at[index, 'Egal-Util'] = (row['p'] - 1) / 9
    cons_df.at[index, 'Util-Egal'] = 1 - cons_df.at[index, 'Egal-Util']

# As of 14/11, using processed_data_one_action_with_factor_2.5_5.0 P values 1.0-1.7 are for the scheme, 1.8 - 10.0 are against
against_scheme = list(np.arange(1.8, 10.0, 0.1))
for_scheme = list(np.arange(1.0, 1.7, 0.1))
against_scheme = [round(num, 1) for num in against_scheme]
for_scheme = [round(num, 1) for num in for_scheme]

final_df = pd.DataFrame()
for consensus in relevent_consensuses:
    filtered_df = cons_df[cons_df['p'] == consensus][relevant_columns]
    filtered_df['decision'] = (filtered_df['Rel_div_p'] * filtered_df['Rel-Nonrel']) + (filtered_df['Nonrel_div_p'] * filtered_df['Nonrel-Rel'])
    final_df = pd.concat([final_df, filtered_df], ignore_index=True)
final_df.to_csv(final_df_savename)
print(final_df.to_string())

       p  Rel-Nonrel  Nonrel-Rel  Rel_div_p  Nonrel_div_p  Egal-Util  Util-Egal  decision
0    1.0    0.346411    0.653589   0.001517      0.028995   0.000000   1.000000  0.019476
1    1.1    0.346043    0.653957  -0.006315      0.028138   0.011111   0.988889  0.016216
2    1.2    0.345798    0.654202  -0.014573      0.022175   0.022222   0.977778  0.009468
3    1.3    0.345695    0.654305  -0.018456      0.018935   0.033333   0.966667  0.006009
4    1.4    0.345776    0.654224  -0.020717      0.017142   0.044444   0.955556  0.004051
5    1.5    0.346117    0.653883  -0.022280      0.016003   0.055556   0.944444  0.002753
6    1.6    0.346774    0.653226  -0.023652      0.015134   0.066667   0.933333  0.001684
7    1.7    0.347730    0.652270  -0.024931      0.014435   0.077778   0.922222  0.000746
8    1.8    0.349271    0.650729  -0.026228      0.013833   0.088889   0.911111 -0.000159
9    1.9    0.350279    0.649721  -0.026999      0.013505   0.100000   0.900000 -0.000682
10   2.0  

In [29]:
# Read in data
agent_csv_file = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/processed_data_one_action_ess.csv_with_factor_2.5_5.0.csv"
df = pd.read_csv(agent_csv_file)
# sanity check
print(df.head())

   Unnamed: 0 country   rel  nonrel  a_div_rel  a_div_nonrel  Rel-Nonrel  \
0           0      AT   642    1216   0.238837      0.561952    0.731701   
1           1      BE   621    1116  -0.319377     -0.271804    0.713731   
2           2      CH   767     666   0.754020      1.000000    0.447139   
3           3      CZ   577    1483   0.106875      0.144976    0.829854   
4           4      DE  1264    1488   0.010549      0.340502    0.561047   

   Nonrel-Rel  
0    0.268299  
1    0.286269  
2    0.552861  
3    0.170146  
4    0.438953  


In [30]:
def binary_principles(df):
    df['decision'] = np.nan  # Initialize the 'decision' column

    for index, row in df.iterrows():
        df.at[index, 'decision'] = (row['rel-nonrel'] * row['a_div_rel']) + (row['nonrel-rel'] * row['a_div_nonrel'])
        if df.at[index, 'decision'] < 0:
            df.at[index, 'egal'] = 1
            df.at[index, 'util'] = 0
        else:
            df.at[index, 'egal'] = 0
            df.at[index, 'util'] = 1
    return df

def range_principles(df):
    """
    This function calculates the decision based on the range of principles
    - factor is calculated as a random p value the agent will hold. 
        The principle will be calculated from principles that are in opposition to the agents real choice
    """
    df['decision'] = np.nan  # Initialize the 'decision' column
    for index, row in df.iterrows():
        total_interviewees = row['rel'] + row['nonrel']

        df.at[index, 'decision'] = (row['Rel-Nonrel'] * row['a_div_rel']) + (row['Nonrel-Rel'] * row['a_div_nonrel'])
        if df.at[index, 'decision'] < 0:
            factor = np.random.choice(against_scheme)
            egal = (factor / 10) * total_interviewees
            util = total_interviewees - egal
            egal = round(egal, 0)
            util = round(util, 0)
            # Factor is 1.0 then [egal, util] = [0, 1]
            # Factor is 10.0 then [egal, util] = [1, 0]
            # Factor is 1.8 then [egal, util] = [0.18, 0.82]

            df.at[index, 'egal'] = egal
            df.at[index, 'util'] = util
        else:
            factor = np.random.choice(for_scheme)
            egal = (factor / 10) * total_interviewees
            util = total_interviewees - egal
            egal = round(egal, 0)
            util = round(util, 0)
            df.at[index, 'egal'] = egal
            df.at[index, 'util'] = util
    return df


In [31]:
df = range_principles(df)
df

,Unnamed: 0,country,rel,nonrel,a_div_rel,a_div_nonrel,Rel-Nonrel,Nonrel-Rel,decision,egal,util
0,0,AT,642,1216,0.238837,0.561952,0.731701,0.268299,0.325528,260.0,1598.0
1,1,BE,621,1116,-0.319377,-0.271804,0.713731,0.286269,-0.305759,730.0,1007.0
2,2,CH,767,666,0.754020,1.000000,0.447139,0.552861,0.890013,201.0,1232.0
3,3,CZ,577,1483,0.106875,0.144976,0.829854,0.170146,0.113357,206.0,1854.0
4,4,DE,1264,1488,0.010549,0.340502,0.561047,0.438953,0.155383,385.0,2367.0
5,5,EE,802,1118,0.361596,0.247466,0.623437,0.376563,0.318619,192.0,1728.0
6,6,ES,567,1143,0.073486,0.281423,0.752632,0.247368,0.124923,171.0,1539.0
7,7,FI,739,1099,-0.493911,-0.028814,0.646899,0.353101,-0.329685,1526.0,312.0
8,8,FR,1063,899,0.105049,0.561735,0.437309,0.562691,0.362022,216.0,1746.0
9,9,GB,541,1287,0.040049,0.252525,0.806072,0.193928,0.081254,219.0,1609.0


In [32]:
# Save the principles a file
principles_df = df[['country', 'egal', 'util']]
principles_df.rename(columns={'egal': 'rel', 'util': 'nonrel'}, inplace=True)

## principles_df will give you just the principles, df will give you the
##  principles and all other data for each agent
principles_df.to_csv("/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/single_example_results/single_example/12-01-2025-principles.csv", index=False)
# df.to_csv("/home/ia23938/Documents/GitHub/ValueSystemsAggregation/process_data/14-11-processed_data_with_principles_ess.csv", index=False)


/tmp/ipykernel_100070/3382265622.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  principles_df.rename(columns={'egal': 'rel', 'util': 'nonrel'}, inplace=True)
